In [1]:
import warnings
from collections import Counter
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


import pickle
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ctgan import CTGANSynthesizer
from sklearn.model_selection import train_test_split
from utils import *

MODELS_PATH = './models'
dataset = 'puma'
seed = 1
np.random.seed(seed)

# A. Load data, preprocess, and calculate accuracy 

In [2]:
X, y, le = read_data(dataset)

ValueError: 'puma' is not binary classification

In [ ]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
categorical_features=[]
preprocessor = get_preprocessor(X, categorical_features)
rf = RandomForestClassifier(n_jobs=-1, random_state=seed)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', rf)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

# B. Plot confidence scores for X_train

In [ ]:
y_prob = rf.predict_proba(X_train)
y_conf_train = y_prob[:, 0]  # confidence scores

plot_confidence_levels(y_conf_train, "Confidence scores for X_train")

# C. Train CTGAN and plot the training loss

In [ ]:
z_features = get_noise_features(X_train, categorical_features)
z_rows = int(0.25 * X_train.shape[0])
z = gen_random_noise(shape=(z_rows, z_features))

batch_size = 50
epochs = 50
confidence_level = 0.9
gen_lr = 2e-5
loss = 'log'

rf_ctgan = CTGANSynthesizer(batch_size=batch_size,
                            blackbox_model=rf,
                            preprocessing_pipeline=preprocessor,
                            bb_loss=loss,
                            verbose=False
                            )

hist = rf_ctgan.fit(train_data=z,
                    epochs=epochs,
                    confidence_level=confidence_level,
                    gen_lr=gen_lr,
                    )

# rf_ctgan.save(f"{MODELS_PATH}/{dataset}_ctgan_c_{confidence_level}.pkl")

plot_losses(hist, title=f'{dataset} loss, c = {confidence_level}')
print()

# D. Plot confidence scores for 100 generated samples 

In [ ]:
# check confidence for the generated samples
samples = 100

gen_data = rf_ctgan.sample(samples)
y_prob = rf.predict_proba(gen_data)
y_conf_gen = y_prob[:, 0]  # confidence scores

plot_confidence_levels(y_conf_gen, f"Scores of generated {samples} samples (c={confidence_level})")

# E. Find generated samples above the confidence score

In [ ]:
# find samples s such that s.confidence > c
indices = np.argwhere(y_conf_gen>confidence_level).squeeze()
print(f"indecies:\n\t{indices}\nconfidence levels:\n\t{y_conf_gen[indices]}")
gen_indices = indices if indices.shape != () else [indices] # avoide no shape arrays

In [ ]:
# inverse the generated data
scaler = get_scaler(preprocessor)
gen_data_above_c_before = gen_data.iloc[gen_indices]
gen_data_above_c = scaler.inverse_transform(gen_data_above_c_before)
gen_data_above_c = pd.DataFrame(gen_data_above_c).set_index(gen_data_above_c_before.index)

# F. Print most similar examples (X_similiar)

In [ ]:
similarities = calc_similarities(gen_data_above_c, X_train)
X_similiar_indices = [el[0] for el in similarities.values()]
print(f"gen_sample_above_c -> (most_similiar_sample_x_train, cosine_score)\n\n{similarities}")

# G. Print confidence scores for X_similiar

In [ ]:
# extract X_similiar 
X_train_pd = pd.DataFrame(X_train)
X_similiar = X_train_pd.iloc[X_similiar_indices]

# print confidence scores
# y_prob_similar = rf.predict_proba(X_similiar)
# y_conf_similar = y_prob_similar[:, 0]
print(f"confidence scores for similar samples:\n{y_conf_train[X_similiar_indices]}")


# Plot as table

In [ ]:
data = []
for gen_idx, value in similarities.items():
    similar_idx = value[0]
    similarity = value[1]
    gen_conf = y_conf_gen[gen_idx]
    similar_conf = y_conf_train[similar_idx]
    data.append([gen_idx, gen_conf, similar_idx, similar_conf, similarity])

columns = ['gen_idx', 'score', 'sim_idx', 'score', 'similarity']
results = pd.DataFrame(data, columns=columns) 
results

# Plot Similarities Distribution 